In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session data=pd.read_csv('train.csv')


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
data=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
data.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# model preprocessing


**count the sum of nan values in train data**

In [3]:
data.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**fill the the nan values of age in train and check the count again**

In [4]:
data.fillna(value = data.median(),inplace = True)
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**count the nan values in test data**

In [5]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

**fill the age **

In [6]:
test['Age'].fillna(value = test['Age'].median(),inplace = True)

In [7]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

**get the class of the person of nan value in fare**

In [8]:
row_of_data=test.loc[test.Fare.isna()== True]
pclass=int(row_of_data['Pclass'])
print(pclass)

3


**fill the fare nan value**

In [9]:
fare_replace = test.loc[test.Pclass == 3].Fare.median()
test['Fare'].fillna(value = fare_replace,inplace = True)


**split the train data into x,y**

In [10]:
y = data.iloc[:, 1].values
x = data.iloc[:,data.columns != 'Survived' ].values

In [11]:
# the variables names
variables = ['PassengerId','Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


# convert x into dataframe
x  = pd.DataFrame(x, columns = variables)

# convert test data into dataframe
test  = pd.DataFrame(test, columns = variables)

**drop the unnescessary columns**

In [12]:
# drop the non useful columns from train data 
x=x.drop(['Name','Cabin','Ticket','PassengerId'], axis = 1)
x=x.drop(['Embarked'], axis = 1)

In [13]:

# drop the non useful columns from test data also
test=test.drop(['Name','Cabin','Ticket','Embarked'], axis = 1)
passengerid=test.iloc[:,0]
test=test.drop(['PassengerId'], axis = 1)

**encode male=1 , female=0 in both test and train data**

In [14]:
x['Sex'].replace(to_replace='male',value=1,inplace=True)
x['Sex'].replace(to_replace='female',value=0,inplace=True)
test['Sex'].replace(to_replace='male',value=1,inplace=True)
test['Sex'].replace(to_replace='female',value=0,inplace=True)

**nitiate the random forrest regression  model  and fit it to the train data**



**split the data for testing the model befor applying it on the real test data**

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# model bulding

In [16]:


from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=10, min_samples_leaf=6, min_samples_split=6,n_estimators=1750)
classifier.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=6, min_samples_split=6,
                       n_estimators=1750)

# model evaluation

**evaluate the accuracy of the model with predicting test data we have from spliting**

In [17]:
y_test_pred = classifier.predict(x_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_test_pred)
print(cm)
accuracy_score(y_test, y_test_pred)

[[94 11]
 [26 48]]


0.7932960893854749

**predict the (survive) column  for kaggle test data**

In [18]:
 
y_pred = classifier.predict(test)

**make a dataframe and add 2 columns : passenger id , survived (predicted)**

In [19]:

predections = pd.DataFrame()
predections['PassengerId']=passengerid
predections['Survived']=y_pred

**save it to csv**


In [20]:
predections.to_csv("pedictions.csv",index= False)